In [6]:
#Instalamos librerias necesarias
!pip install opencv-python mediapipe gradio -q


In [7]:
# Importamos paquetes necesarios
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import gradio as gr
import tempfile
import os

In [ ]:
def eye_aspect_ratio(ojo):
    # Calculo de la distancia euclidiana entre los dos conjuntos de coordenadas verticales
    A = dist.euclidean(ojo[1], ojo[5])
    B = dist.euclidean(ojo[2], ojo[4])
    # Calculo de la distancia euclidiana entre las coordenadas horizontales
    C = dist.euclidean(ojo[0], ojo[3])
    # Calculo de la relación de aspecto del ojo
    ear = (A + B) / (2.0 * C)

    return ear

# Constantes
EAR_UMBRAL = 0.3
EAR_FRAMES = 5

# Inicializacion de contador de cuadros y alarma
CONTADOR = 0
ALARMA = False

# Inicializacion de MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Índices para los puntos de referencia de los ojos (ajustar si es necesario)
INDICE_OJO_IZQ = [362, 385, 387, 263, 373, 380]
INDICE_OJO_DER = [33, 160, 158, 133, 153, 144]


def procesamiento_video(video_path):
    global CONTADOR, ALARMA, INDICE_OJO_IZQ, INDICE_OJO_DER
    CONTADOR = 0
    ALARMA = False

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error al abrir el archivo")
        return

    # propiedades del video original
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # para construcción de video de salida
    temp_output_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)
    temp_output_path = temp_output_file.name
    temp_output_file.close()

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(temp_output_path, fourcc, fps, (width, height))


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Conversion a RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Proceso con MediaPipe Face Mesh
        resultado = face_mesh.process(frame_rgb)

        if resultado.multi_face_landmarks:
            for face_landmarks in resultado.multi_face_landmarks:
                # Dibujo de malla facial
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=drawing_spec)

                # Extraccion de ojos
                landmarks = np.array([(lm.x * width, lm.y * height) for lm in face_landmarks.landmark])
                landmarks_ojo_izq = landmarks[INDICE_OJO_IZQ]
                landmarks_ojo_der = landmarks[INDICE_OJO_DER]

                # Calculo de ear de ambos ojos
                izq_ear = eye_aspect_ratio(landmarks_ojo_izq)
                der_ear = eye_aspect_ratio(landmarks_ojo_der)

                # Average de ear
                ear = (izq_ear + der_ear) / 2.0

                # Chequeo de UMBRAL
                if ear < EAR_UMBRAL:
                    CONTADOR += 1
                    # si ojos permanecen cerrados duarnte mayor tiempo que los indicados
                    # se muestra alarma
                    if CONTADOR >= EAR_FRAMES:
                        # si alarma esta desactivada, se cambia estado
                        if not ALARMA:
                            ALARMA = True

                        # Alarma en el frame
                        cv2.putText(frame, "ALERTA DE SOMNOLENCIA!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                # reseteo de CONTADOR cambio estado de alarma
                else:
                    CONTADOR = 0
                    ALARMA = False

                # dibujo de ear en el frame
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        out.write(frame)

    cap.release()
    out.release()

    return temp_output_path

# creacion de interfaz con Gradio
iface = gr.Interface(
    fn=procesamiento_video,
    inputs=gr.Video(label="Subir Video"),
    outputs=gr.Video(label="Video Procesado"),
    title="Detección de Somnolencia usando MediaPipe",
    description="Cargar video para detectar somnolencia en base a calculo de apertura de ojos."
)

iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e214f9e8dad51de511.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> None
Killing tunnel 127.0.0.1:7861 <> https://e214f9e8dad51de511.gradio.live
